In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sqlalchemy import create_engine

In [2]:
# import the first dataset 2019_data
golf_file = "../Resources/2019_data.csv"
golf_df = pd.read_csv(golf_file)
golf_df.head()

,Player Name,Date,Statistic,Variable,Value
0,Cameron Champ,2019-08-04,Driving Distance,Driving Distance - (ROUNDS),70
1,Luke List,2019-08-04,Driving Distance,Driving Distance - (ROUNDS),62
2,Rory McIlroy,2019-08-04,Driving Distance,Driving Distance - (ROUNDS),60
3,Wyndham Clark,2019-08-04,Driving Distance,Driving Distance - (ROUNDS),79
4,Bubba Watson,2019-08-04,Driving Distance,Driving Distance - (ROUNDS),63


In [3]:
# Use pd.datetimeindex to convert field to date
golf_df['Date'] = pd.DatetimeIndex(golf_df['Date']).year
golf_df.head()

,Player Name,Date,Statistic,Variable,Value
0,Cameron Champ,2019,Driving Distance,Driving Distance - (ROUNDS),70
1,Luke List,2019,Driving Distance,Driving Distance - (ROUNDS),62
2,Rory McIlroy,2019,Driving Distance,Driving Distance - (ROUNDS),60
3,Wyndham Clark,2019,Driving Distance,Driving Distance - (ROUNDS),79
4,Bubba Watson,2019,Driving Distance,Driving Distance - (ROUNDS),63


In [4]:
#import the second dataset 
golf_historical = "../Resources/PGA_Data_Historical.csv"
historical_df = pd.read_csv(golf_historical)
historical_df.head()

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


In [5]:
#create filtered df from specific columns
golf_cols = ["Player Name","Date","Statistic","Value"]
golf_transformed = golf_df[golf_cols]
golf_transformed['Value'] = golf_transformed['Value'].str.extract('([-+]?\d*\.\d+|\d+)').astype(float)

#rename the column headers
golf_transformed = golf_transformed.rename(columns={"Player Name":'Name',
                                                   "Date":'Date',
                                                   "Statistic":'Skill',
                                                   "Value":'Rank'})

golf_transformed['Rank'] = golf_transformed['Rank'].replace(',','')
#clean the data by dropping duplicates and setting the index
#golf_transformed.drop_duplicates("Name",inplace = True)

golf_transformed.head()

C:\Users\yujtang\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Name,Date,Skill,Rank
0,Cameron Champ,2019,Driving Distance,70.0
1,Luke List,2019,Driving Distance,62.0
2,Rory McIlroy,2019,Driving Distance,60.0
3,Wyndham Clark,2019,Driving Distance,79.0
4,Bubba Watson,2019,Driving Distance,63.0


In [6]:
#create filtered df from specific columns
history_cols = ["Player Name","Season","Statistic","Value"]
history_transformed = historical_df[history_cols]
#history_transformed['Value'] = history_transformed['Value'].str.replace(',','').str.replace('T','').str.replace(' ','')\
#    .str.replace('$','').astype(float)
history_transformed['Value'] = history_transformed['Value'].str.extract('([-+]?\d*\.\d+|\d+)').astype(float)

#rename the column headers
history_transformed = history_transformed.rename(columns={"Player Name":'Name',
                                                   "Season":'Date',
                                                   "Statistic":'Skill',
                                                   "Value":'Rank'})

#clean the data by dropping duplicates and setting the index
#history_transformed.drop_duplicates("Name",inplace = True)

history_transformed.head()

C:\Users\yujtang\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Name,Date,Skill,Rank
0,Robert Garrigus,2010,Driving Distance,71.0
1,Bubba Watson,2010,Driving Distance,77.0
2,Dustin Johnson,2010,Driving Distance,83.0
3,Brett Wetterich,2010,Driving Distance,54.0
4,J.B. Holmes,2010,Driving Distance,100.0


In [7]:
# Create database connections
connection_string = "postgres:940518Tang@localhost:5432/golf_db"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
engine.table_names()

['golf', 'history']

In [9]:
golf_transformed.to_sql(name='golf', con=engine, if_exists='append', index=True, index_label="ID")

In [10]:
history_transformed.to_sql(name='history', con=engine, if_exists='append', index=True, index_label="ID")